In [37]:
import sys
sys.path.append('..')


import numpy as np
import h5py
from lib.Utilities import *
import os
from collections import Counter

In [39]:
datafile = '/storage/ms5267@drexel.edu/precicecap_downloads/90_Patient_2023-03-21_12:19.h5'

annotation_group_names = []

# Directory containing the HDF5 files
directory_path = '/storage/ms5267@drexel.edu/precicecap_downloads/'

for filename in os.listdir(directory_path):
	if filename.endswith('.h5'):
		log_info(f"Processing {filename}")
		datafile = os.path.join(directory_path, filename)
		with h5py.File(datafile, 'r') as file:
			annotations_group = file['Annotations']
			for name, subgroup in annotations_group.items():
				annotation_data = file[f'Annotations/{name}'][:]
				if len(annotation_data)>0:
					annotation_group_names.extend(list(annotation_data[:, -2]))
				else:
					log_info("No annotation group")

print(Counter(annotation_group_names))

14:30:36 :	  Processing 85_Patient_2023-05-12_17:53.h5 

14:30:36 :	  Processing 59_Patient_2022-01-31_23:19.h5 

14:30:36 :	  Processing 74_Patient_2023-08-05_06:00.h5 

14:30:36 :	  Processing 110_Patient_2023_Sep_28__23_52_07_705708.h5 

14:30:36 :	  Processing 90_Patient_2023-03-21_19:57.h5 

14:30:36 :	  No annotation group 

14:30:36 :	  Processing 4_Patient_2022-02-05_08:59.h5 

14:30:36 :	  Processing 73_Patient_2017_Dec_18__11_19_55_297272.h5 

14:30:36 :	  Processing 34_Patient_2023-04-04_22:31.h5 

14:30:36 :	  Processing 53_Patient_2023-06-25_21:39.h5 

14:30:36 :	  Processing 101_Patient_2023_Nov_9__22_24_41_155873.h5 

14:30:36 :	  Processing 90_Patient_2023-03-21_12:19.h5 

14:30:36 :	  Processing 50_Patient_2023-06-12_21:10.h5 

14:30:36 :	  Processing 35_Patient_2023-04-03_19:51.h5 

14:30:36 :	  Processing 55_Patient_2023-06-13_00:47.h5 

14:30:36 :	  Processing 139_Patient_2024_Mar_4__7_32_51_662674.h5 

14:30:36 :	  Processing 34_Patient_2023-04-05_12:23.h5 

14:30:

In [94]:

# Directory containing the HDF5 files
directory_path = '/storage/ms5267@drexel.edu/precicecap_downloads/'

for filename in os.listdir(directory_path):
	if filename.endswith('.h5'):
		log_info(f"Processing {filename}")
		datafile = os.path.join(directory_path, filename)

		with h5py.File(datafile, 'r') as file:
			# First get the timestamp list for abp and ecg
			try:
				abp_timestamp = file['Waveforms/ART_na_Timestamps']
			except KeyError:
				abp_timestamp = file['Waveforms/ABP_na_Timestamps']	
			
			ecg_timestamp = file['Waveforms/ECG_II_Timestamps']
			abp_timestamp = abp_timestamp[:]
			ecg_timestamp = ecg_timestamp[:]

			annotations_group = file['Annotations']
			

			for name, subgroup in annotations_group.items():
				annotation_data = file[f'Annotations/{name}'][:]

				# If no annotation data need to print that there is no annotation for this file.
				if len(annotation_data)>0:
					annotation_group_names.extend(list(annotation_data[:, -2]))
				else:
					log_info(f"No annotation group in {filename}")

				mask =  (annotation_data[:, -2] == b'ECG') | \
						(annotation_data[:, -2] == b'ABP') | \
						(annotation_data[:, -2] == b'ART') | \
						(annotation_data[:, -2] == b'ART1') | \
						(annotation_data[:, -2] == b'ART2')

				ecg_abp_annotations = annotation_data[mask]
			
			idx_list = []
			for ann in ecg_abp_annotations:
				ann_start_ts = float(ann[2])*1e3
				ann_end_ts = float(ann[3])*1e3
				# print(ann_start_ts, ann_end_ts, ann[4])
				if ann[4]==b'ECG':
					ann_start_idx, ann_end_idx = find_idx_from_ts(ecg_timestamp, ann_start_ts, ann_end_ts)
				else:
					ann_start_idx, ann_end_idx = find_idx_from_ts(abp_timestamp, ann_start_ts, ann_end_ts)

				idx_list.append([ann_start_idx, ann_end_idx])

			ann_array = np.hstack((np.array(idx_list), ecg_abp_annotations))
			log_info(ann_array.shape)
			np.savetxt('../data/annotations/filename-annotations.csv', ann_array, delimiter=',', fmt='%s')

15:23:04 :	  Processing 85_Patient_2023-05-12_17:53.h5 

15:23:10 :	  (1002, 8) 

15:23:10 :	  Processing 59_Patient_2022-01-31_23:19.h5 

15:23:10 :	  (620, 8) 

15:23:10 :	  Processing 74_Patient_2023-08-05_06:00.h5 

15:23:17 :	  (1219, 8) 

15:23:17 :	  Processing 110_Patient_2023_Sep_28__23_52_07_705708.h5 

15:23:23 :	  (3756, 8) 

15:23:24 :	  Processing 90_Patient_2023-03-21_19:57.h5 

15:23:26 :	  No annotation group 



IndexError: index -2 is out of bounds for axis 1 with size 0

In [86]:
abp_timestamp[4979380], abp_timestamp[4985252]

(1680689974005418, 1680690021002149)

In [56]:
abp_timestamp

array([1680650063562978, 1680650063570981, 1680650063578985, ...,
       1680697542722857, 1680697542730861, 1680697542738865])

In [76]:
find_idx_from_ts(abp_timestamp, 1680647558000000, 1680947589000000)

(0, 5923743)

In [78]:
min(ecg_timestamp)

1680647498287486